In [7]:
import json
import os

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Create the dictionaries directory
dict_dir = '/content/drive/MyDrive/FedComs/Dictionaries'
if not os.path.exists(dict_dir):
    os.makedirs(dict_dir)

# Define the labor market indicators dictionary
LABOR_INDICATORS = {
    "General Labor": ["labor", "maximum employment", "strike"],
    "Employment": ["employed", "employment", "employment-to-population", "employment to population", "employment/population", "e/p ratio", "employment-population ratio"],
    "Unemployment": ["unemployment", "unemployed", "jobless", "joblessness", "u-3", "u3 rate", "job seeker", "jobseeker", "job seekers", "jobseekers", "want a job", "out of work", "seeking work", "job search", "can't find work", "can\'t find a job"],
    "Participation": ["participation", "labor supply", "marginally attached", "discouraged workers", "retirement", "labor force", "want a job", "labor force participation", "participation rate", "lfpr", "lfp", "labor market participation", "workforce", "early retirement"],
    "Wages": ["pay", "eci", "labor cost", "labor costs", "wage", "wages", "wage growth", "wage inflation", "worker compensation", "earnings", "salary", "salaries", "hourly earnings", "labor compensation"],
    "Vacancies": ["vacancy", "vacancies", "labor demand", "labor shortage", "job ads", "want ads", "job postings", "job opening", "job openings", "jolts", "unfilled position", "open position", "vacant job", "vacancy rate", "job vacancy", "posted opening", "help wanted", "available job", "job posting", "position opening", "vacancy yield", "beveridge curve", "job advertisement", "job availability", "hiring difficulty"],
    "Quits": ["quit", "quits", "resignation", "resignations", "voluntary separation", "turnover", "voluntary turnover", "job changing", "job switching", "job hopping", "job-to-job", "worker turnover", "leaving jobs"],
    "Layoffs": ["layoff", "layoffs", "involuntary separation", "dismissal", "dismissals", "discharged", "discharge", "downsizing", "redundancy", "workforce reduction", "job cuts", "reduction in force", "rif", "termination", "fired", "job loss", "permanent layoff", "temporary layoff", "furlough", "mass layoff", "involuntary job loss", "job destruction", "establishment closure", "job shedding", "plant closure", "business closure", "pink slip", "staff reduction", "displaced workers"],
    "Hiring": ["hiring", "hire", "hires", "recruitment", "new jobs", "job gains", "job finding", "payroll", "nonfarm payroll", "job gain", "job growth", "employment gain", "employment growth", "job creation", "add to payrolls"]
}

# Define the inflation indicators dictionary
INFLATION_INDICATORS = {
    "General Inflation": {
        "general_patterns": [
            r"\bdeflation\b",
            r"\bcosts?\b",
            r"\bprices?\b",
            r"\bprice\s+stability\b",
            r"\bprices\b(?!\s+(of|in|for|on)\s+(services?|goods|housing|energy|food|consumer|commodity|raw\s+materials?|oil|gasoline|fuel|electricity|natural\s+gas|shelter|rent|rental|residential))"
        ]
    },
    "Core Measures": {
        "core_cpi_patterns": [
            r"\b(core|underlying)\s+cpi\b",
            r"\bcpi\s+(core|underlying)\b",
            r"\b(core|underlying)\s+consumer\s+price\s+index\b",
            r"\b(consumer\s+price\s+index)\s+(core|underlying)\b",
            r"\bcore\s+consumer\s+price\s+inflation\b"
        ],
        "core_general_patterns": [
            r"\bcore\s+inflation\b",
            r"\bunderlying\s+inflation\b",
            r"\bunderlying\s+price\s+pressures?\b",
            r"\bcore\s+prices?\b",
            r"\bcore\s+price\s+inflation\b"
        ],
        "core_pce_patterns": [
            r"\b(core|underlying)\s+(pce|personal\s+consumption\s+expenditures?)\b(?!\s+real)",
            r"\b(pce|personal\s+consumption\s+expenditures?)\s+(core|underlying)\b(?!\s+real)",
            r"\bcore\s+personal\s+consumption\b(?!\s+expenditures?\s+real)",
            r"(?<!real\s)\b(core|underlying)\s+(pce|personal\s+consumption\s+expenditures?)\b"
        ],
        "excluding_patterns": [
            r"\b(cpi|pce|consumer\s+price|personal\s+consumption|inflation|prices?)\s+(excluding|ex|minus|without|less)\s+(food|energy|volatile)\b(?!\s+real)",
            r"\b(excluding|ex|minus|without|less)\s+(food|energy|volatile)\s+(cpi|pce|consumer\s+price|inflation|prices?)\b(?!\s+real)",
            r"\b(cpi|pce|inflation|prices?)\s+(excluding|ex|minus|without|less)\s+(food\s+and\s+energy|volatile\s+items)\b(?!\s+real)",
            r"\b(stripping\s+out|stripped\s+of)\s+(food|energy|volatile)\b",
            r"\bex[\-\s]food\s+and\s+energy\b",
            r"\bexcluding\s+volatile\s+(food\s+and\s+)?energy\b"
        ],
        "trimmed_patterns": [
            r"\b(trimmed|weighted)\s+(mean|median|average)\s+(cpi|pce|inflation|prices?)\b",
            r"\b(cpi|pce|inflation|prices?)\s+(trimmed|weighted)\s+(mean|median|average)\b",
            r"\b(trimmed-mean|weighted-median)\b",
            r"\btrimmed\s+mean\b"
        ],
        "excluding_housing_patterns": [
            r"\b(services?|cpi|pce|inflation|prices?)\s+(excluding|ex|minus|without|less)\s+(housing|shelter|rent)\b",
            r"\b(excluding|ex|minus|without|less)\s+(housing|shelter|rent)\s+(services?|inflation|prices?)\b",
            r"\bex[\-\s]housing\b",
            r"\bexcluding\s+housing\b"
        ],
        "excluding_energy_patterns": [
            r"\b(cpi|pce|inflation|prices?)\s+(excluding|ex|minus|without|less)\s+energy\b",
            r"\b(excluding|ex|minus|without|less)\s+energy\s+(cpi|pce|inflation|prices?)\b",
            r"\bex[\-\s]energy\b",
            r"\bexcluding\s+energy\b"
        ],
        "supercore_patterns": [
            r"\bsupercore\b",
            r"\bsuper\s+core\b",
            r"\bservices?\s+(excluding|ex|minus|without)\s+(housing|shelter)\b",
            r"\bcore\s+services?\s+(excluding|ex|minus|without)\s+(housing|shelter)\b",
            r"\bcore\s+services\s+ex[\-\s]housing\b"
        ]
    },
    "Headline Measures": {
        "headline_cpi_patterns": [
            r"\b(headline|overall|total|all-items?)\s+cpi\b",
            r"\bcpi\s+(headline|overall|total)\b",
            r"\b(headline|overall|total|all-items?)\s+consumer\s+price\s+index\b",
            r"\b(headline|overall|total|all-items?)\s+consumer\s+prices?\b",
            r"\b(consumer\s+price\s+index|consumer\s+prices?)\s+(headline|overall|total)\b",
            r"\bcpi\b(?!\s+(core|excluding|ex|minus|without|trimmed))",
            r"\bconsumer\s+price\s+index\b(?!\s+(core|excluding|ex|minus|without))",
            r"\ball\s+items\s+cpi\b",
            r"\btotal\s+cpi\b"
        ],
        "headline_general_patterns": [
            r"\bheadline\s+inflation\b",
            r"\boverall\s+inflation\b",
            r"\btotal\s+inflation\b",
            r"\ball[\-\s]items?\s+inflation\b",
            r"\bheadline\s+prices?\b",
            r"\boverall\s+prices?\b",
            r"\broad\s+prices?\b", r"\broad\s+inflation\b", r"inflation\s+broadly\b"
        ],
        "headline_pce_patterns": [
            r"\b(headline|overall|total|all-items?)\s+pce\b(?!\s+real)",
            r"\bpce\s+(headline|overall|total)\b(?!\s+real)",
            r"\b(headline|overall|total|all-items?)\s+personal\s+consumption\s+expenditures?\b(?!\s+real)",
            r"\b(pce|personal\s+consumption\s+expenditures?)\s+(headline|overall|total)\b(?!\s+real)",
            r"\bpce\b(?!\s+(core|excluding|ex|minus|without|real))",
            r"\bpersonal\s+consumption\s+expenditures?\b(?!\s+(core|excluding|ex|minus|without|real))",
            r"\ball\s+items\s+pce\b(?!\s+real)",
            r"\btotal\s+pce\b(?!\s+real)",
            r"(?<!real\s)\b(headline|overall|total|all-items?)\s+(pce|personal\s+consumption\s+expenditures?)\b"
        ],
        "general_headline_patterns": [
            r"\bheadline\s+inflation\b",
            r"\boverall\s+inflation\b",
            r"\btotal\s+inflation\b",
            r"\ball[\-\s]items?\s+inflation\b"
        ]
    },
    "Sectoral Measures": {
        "services_patterns": [
            r"\bservices?\s+(inflation|prices?|costs?|pricing)\b",
            r"\b(inflation|prices?|costs?|pricing)\s+(in|of|for|on)\s+services?\b",
            r"\b(price|cost|pricing)\s+of\s+services?\b",
            r"\bservice\s+(sector\s+)?inflation\b",
            r"\bservices\s+component\b"
        ],
        "goods_patterns": [
            r"\b(consumer\s+)?goods\s+(inflation|prices?|costs?|pricing)\b",
            r"\b(inflation|prices?|costs?|pricing)\s+(in|of|for|on)\s+(consumer\s+)?goods\b",
            r"\b(price|cost|pricing)\s+of\s+(consumer\s+)?goods\b",
            r"\bgoods\s+(sector\s+)?inflation\b",
            r"\bgoods\s+component\b"
        ],
        "housing_patterns": [
            r"\bhousing\s+(inflation|prices?|costs?|pricing)\b",
            r"\b(inflation|prices?|costs?|pricing)\s+(in|of|for|on)\s+housing\b",
            r"\b(price|cost|pricing)\s+of\s+housing\b",
            r"\bhousing\s+(sector\s+)?inflation\b",
            r"\bhousing\s+component\b",
            r"\b(shelter|rent|rental|residential)\s+(inflation|prices?|costs?)\b",
            r"\bowners?\s+equivalent\s+rent\b",
            r"\boer\b",
            r"\brent\s+inflation\b"
        ],
        "energy_patterns": [
            r"\benergy\s+(inflation|prices?|costs?|pricing)\b",
            r"\b(inflation|prices?|costs?|pricing)\s+(in|of|for|on)\s+energy\b",
            r"\b(gasoline|fuel|electricity|natural\s+gas)\s+(inflation|prices?|costs?)\b"
        ],
        "food_patterns": [
            r"\bfood\s+(inflation|prices?|costs?|pricing)\b",
            r"\b(inflation|prices?|costs?|pricing)\s+(in|of|for|on)\s+food\b",
            r"\bfood\s+component\b"
        ]
    },
    "Producer Price Index": {
        "ppi_patterns": [
            r"\bproducer\s+price\s+index\b",
            r"\bppi\b",
            r"\bproducer\s+(inflation|prices?|costs?)\b",
            r"\bwholesale\s+(inflation|prices?)\b"
        ]
    },
    "Wage Inflation": {
        "wage_patterns": [
            r"\b(wage|wages|salary|salaries|pay|earnings|compensation)\s+(inflation|growth|gains?|increases?|rises?)\b",
            r"\bwage\s+(inflation|growth|gains?|increases?|pressures?)\b",
            r"\baverage\s+hourly\s+earnings\b",
            r"\bemployment\s+cost\s+index\b",
            r"\bunit\s+labor\s+costs?\b",
            r"\blabor\s+costs?\b",
            r"\bwage\s+pressures?\b"
        ]
    },
    "Inflation Expectations": {
        "expectations_patterns": [
            r"\binflation\s+expectations?\b",
            r"\bexpected\s+inflation\b",
            r"\binflation\s+(forecasts?|outlook|projections?)\b",
            r"\b(long-term|short-term)\s+(inflation\s+)?expectations?\b",
            r"\bbreakeven\s+(inflation|rates?)\b",
            r"\binflation\s+compensation\b",
            r"\bfive-year\s+forward\b"
        ]
    },
    "Commodity Prices": {
        "commodity_patterns": [
            r"\bcommodity\s+(inflation|prices?|costs?)\b",
            r"\boil\s+(prices?|costs?)\b",
            r"\benergy\s+(prices?|costs?)\b",
            r"\braw\s+materials?\s+(prices?|costs?)\b"
        ]
    }
}

# Mapping for inflation patterns
INFLATION_PATTERN_TO_INDICATOR = {
    "general_patterns": "General_Inflation",
    "core_cpi_patterns": "Core_CPI",
    "core_general_patterns": "Core",
    "core_pce_patterns": "Core_PCE",
    "excluding_patterns": "Core_CPI",
    "trimmed_patterns": "Core_CPI",
    "excluding_housing_patterns": "Core_CPI",
    "excluding_energy_patterns": "Core_CPI",
    "supercore_patterns": "Core_PCE",
    "headline_cpi_patterns": "Headline_CPI",
    "headline_general_patterns": "Headline",
    "headline_pce_patterns": "Headline_PCE",
    "services_samples": "Services",
    "goods_patterns": "Goods",
    "housing_patterns": "Housing",
    "energy_patterns": "Energy",
    "food_patterns": "Food",
    "ppi_patterns": "PPI",
    "wage_patterns": "Wage_Inflation",
    "expectations_patterns": "Inflation_Expectations",
    "commodity_patterns": "Commodity_Prices"
}

# Save labor indicators
with open(os.path.join(dict_dir, 'labor_indicators.json'), 'w') as f:
    json.dump(LABOR_INDICATORS, f, indent=2)

# Save inflation indicators
with open(os.path.join(dict_dir, 'inflation_indicators.json'), 'w') as f:
    json.dump(INFLATION_INDICATORS, f, indent=2)

# Save inflation pattern mapping
with open(os.path.join(dict_dir, 'inflation_pattern_mapping.json'), 'w') as f:
    json.dump(INFLATION_PATTERN_TO_INDICATOR, f, indent=2)

print("Dictionaries saved successfully!")
print(f"Location: {dict_dir}")
print("Files created:")
print("  - labor_indicators.json")
print("  - inflation_indicators.json")
print("  - inflation_pattern_mapping.json")

Mounted at /content/drive
Dictionaries saved successfully!
Location: /content/drive/MyDrive/FedComs/Dictionaries
Files created:
  - labor_indicators.json
  - inflation_indicators.json
  - inflation_pattern_mapping.json
